In [1]:
import numpy as np
import h5py
from scipy.fft import fftn, ifftn

def simulate_3d_freefall(
	x_min=-20, x_max=20, Nx=128,
	y_min=-20, y_max=20, Ny=128,
	z_min=0,   z_max=40, Nz=128,
	n_steps=100, dt=0.01,
	hbar=1.0, m=1.0, g=9.81,
	# Gaussian floor parameters:
	floor_center_z=0.0,       # plane location
	floor_width=0.5,          # thickness
	floor_strength=50.0,      # barrier height
	# initial wavepacket
	x0=0., y0=0., z0=30.,
	sigma=1.0
):
	"""
	Split-operator in 3D: free-fall under gravity + Gaussian floor at z=0.
	Returns nothing, but writes |psi|^2 into an HDF5 file "probs.h5".
	"""

	# 1) grids
	x = np.linspace(x_min, x_max, Nx, endpoint=False)
	y = np.linspace(y_min, y_max, Ny, endpoint=False)
	z = np.linspace(z_min, z_max, Nz, endpoint=False)
	dx, dy, dz = x[1]-x[0], y[1]-y[0], z[1]-z[0]
	X, Y, Z = np.meshgrid(x, y, z, indexing='ij')  # shape (Nx,Ny,Nz)

	# 2) potentials
	V_grav  = m * g * Z
	V_floor = floor_strength * np.exp(-((Z - floor_center_z)**2)/(2*floor_width**2))
	V_total = V_grav + V_floor

	# 3) initial psi: real Gaussian, at rest
	norm = (1/(2*np.pi*sigma**2))**(3/4)
	psi = norm * np.exp(-((X-x0)**2 + (Y-y0)**2 + (Z-z0)**2)/(4*sigma**2))
	psi = psi.astype(np.complex128)

	# 4) momentum grids & kinetic phase
	kx = 2*np.pi * np.fft.fftfreq(Nx, d=dx)
	ky = 2*np.pi * np.fft.fftfreq(Ny, d=dy)
	kz = 2*np.pi * np.fft.fftfreq(Nz, d=dz)
	KX, KY, KZ = np.meshgrid(kx, ky, kz, indexing='ij')
	kin_phase = np.exp(-1j * dt * hbar * (KX**2 + KY**2 + KZ**2)/(2*m))

	# 5) potential half-step
	pot_phase = np.exp(-1j * V_total * dt/(2*hbar))

	# 6) prepare HDF5 with a context manager
	with h5py.File("probs_3d_test.h5", "w") as f:
		dset = f.create_dataset(
			"prob",
			shape=(n_steps, Nx, Ny, Nz),
			dtype=np.float32,
			compression="gzip", compression_opts=4
		)

		for i in range(n_steps):
			# half-step in V
			psi *= pot_phase
			# full T
			psi = ifftn(kin_phase * fftn(psi, workers=-1), workers=-1)
			# half-step in V
			psi *= pot_phase

			# store probability density
			dset[i, ...] = (np.abs(psi)**2).astype(np.float32)
	# file is closed automatically here


In [4]:
params = {
	'x_min':-20, 'x_max':20, 'Nx':512,
	'y_min':-20, 'y_max':20, 'Ny':512,
	'z_min':-20,   'z_max':20, 'Nz':512,
	'n_steps':600, 'dt':0.01,
	'hbar':1.0, 'm':1.0, 'g':9.81,
	
	# Gaussian floor parameters:
	'floor_center_z':0.0,       # plane location
	'floor_width':0.5,          # thickness
	'floor_strength':25.0,      # barrier height

	# initial wavepacket
	'x0':0., 'y0':0., 'z0':15.,
	'sigma':15.0
}

In [5]:
simulate_3d_freefall(**params)

OSError: [Errno 28] Can't synchronously write data (file write failed: time = Sat May  3 09:31:38 2025
, filename = 'probs_3d_test.h5', file descriptor = 73, errno = 28, error message = 'No space left on device', buf = 0x150120000, total write size = 534190, bytes this sub-write = 534190, offset = 33300225909)